In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [2]:
image_directory = 'sorted_images/'

In [3]:
dataset = []
label = []
point_like_corneal_ulcers = os.listdir(os.path.join(image_directory, '0'))
point_flaky_mixed_corneal_ulcers = os.listdir(os.path.join(image_directory, '1'))
flaky_corneal_ulcers = os.listdir(os.path.join(image_directory, '2'))


In [4]:
INPUT_SIZE = 128
for img_name in point_like_corneal_ulcers:
    img_path = os.path.join(image_directory, '0', img_name)
    img = cv2.imread(img_path)
    if img is not None:
        img = cv2.resize(img, (INPUT_SIZE, INPUT_SIZE))  
        dataset.append(img)
        label.append(0)
        
for img_name in point_flaky_mixed_corneal_ulcers:
    img_path = os.path.join(image_directory, '1', img_name)
    img = cv2.imread(img_path)
    if img is not None:
        img = cv2.resize(img, (INPUT_SIZE, INPUT_SIZE)) 
        dataset.append(img)
        label.append(1)
        
for img_name in flaky_corneal_ulcers:
    img_path = os.path.join(image_directory, '2', img_name)
    img = cv2.imread(img_path)
    if img is not None:
        img = cv2.resize(img, (INPUT_SIZE, INPUT_SIZE)) 
        dataset.append(img)
        label.append(2)
        

In [5]:
dataset = np.array(dataset)
label = np.array(label)

In [6]:
num_samples, height, width, channels = dataset.shape
dataset_flat = dataset.reshape((num_samples, height * width * channels))


In [7]:
x_train, x_test, y_train, y_test = train_test_split(dataset_flat, label, test_size=0.2, random_state=42)


In [8]:
n_components = 64  
pca = PCA(n_components=n_components, whiten=True).fit(x_train)


In [9]:
x_train_pca = pca.transform(x_train)
x_test_pca = pca.transform(x_test)


In [10]:
scaler = StandardScaler().fit(x_train_pca)
x_train_pca = scaler.transform(x_train_pca)
x_test_pca = scaler.transform(x_test_pca)


In [11]:
clf = SVC(C=1.0, kernel='rbf')
clf.fit(x_train_pca, y_train)


SVC()

In [12]:
y_pred = clf.predict(x_test_pca)
accuracy_pca = accuracy_score(y_test, y_pred)
print(f'Accuracy using PCA: {accuracy_pca}')


Accuracy using PCA: 0.6853146853146853
